<a href="https://colab.research.google.com/github/SaeedehHeydarian/PromptEngineering/blob/main/ollama_and_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update && apt install -y pciutils lshw

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
54 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as re

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [ ]:
!ollama run llama3.1

Error: ollama server not responding - could not connect to ollama server, run 'ollama serve' to start it


In [ ]:
!pip install -U langchain-ollama

In [ ]:
from langchain_ollama import ChatOllama


In [ ]:
model = ChatOllama(
    model="llama3.1",
    temperature=1,

)

In [ ]:
model.invoke("Do you train on persian text?")

AIMessage(content="I have been trained on a significant amount of text data in multiple languages, including Persian (also known as Farsi or Tajik). However, the quality and quantity of Persian text data may not be as extensive as that for more widely spoken languages like English.\n\nThat being said, I can still understand and respond to questions and engage in conversations in Persian. Keep in mind that:\n\n1. **Language nuances**: My proficiency in Persian might not be on par with native speakers or highly advanced language models.\n2. **Vocabulary limitations**: While I have been trained on a large corpus of text data, the vocabulary in my training dataset may still be limited compared to other languages.\n\nIf you'd like to converse in Persian or need help with understanding a specific topic in Farsi, feel free to ask, and I'll do my best to assist you!", additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-11-27T11:12:31.032365351Z', 'done': True, 'do

بخش اول: دانلود مدل embedding


In [ ]:
!pip install langchain langchain-huggingface langchain_community faiss-cpu

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings


In [ ]:
model_name = "HooshvareLab/bert-base-parsbert-uncased"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf_embedding = HuggingFaceEmbeddings(
model_name=model_name,
model_kwargs=model_kwargs,
encode_kwargs=encode_kwargs
)


In [ ]:
text_1="ما در هوشواره معتقدیم با انتقال صحیح دانش و آگاهی، همه افراد میتوانند از ابزارهای هوشمند استفاده کنند. شعار ما هوش مصنوعی برای همه اس"

In [ ]:
embed_1= hf_embedding.embed_query(text_1)


بخش دوم: ساخت vectorstore


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader


In [ ]:
texts=["درد قفسهٔ سینه می‌تواند ناشی از عوامل مختلفی مانند استرس، مشکلات عضلانی یا بیماری‌های قلبی باشد",
       "سطح قند خون بیماران باید به‌طور منظم اندازه‌گیری و ثبت شود",
       "فعالیت بدنی روزانه نقش مهمی در بهبود سلامت قلب و عروق دارد.",
       "مصرف صحیح و منظم داروها طبق نسخهٔ پزشک برای اثربخشی درمان ضروری است."]

In [ ]:
db=FAISS.from_texts(texts,hf_embedding)

In [ ]:
result=db.similarity_search('بیماری قلبی' , k=2)

In [ ]:
result

[Document(id='fcfdea6d-dc10-4160-a54e-bac1748b43da', metadata={}, page_content='درد قفسهٔ سینه می\u200cتواند ناشی از عوامل مختلفی مانند استرس، مشکلات عضلانی یا بیماری\u200cهای قلبی باشد'),
 Document(id='ab0ffa33-eeeb-433f-8891-5ee717d03289', metadata={}, page_content='فعالیت بدنی روزانه نقش مهمی در بهبود سلامت قلب و عروق دارد.')]

In [ ]:
db.similarity_search_with_relevance_scores('اضطراب' , k=2)

/tmp/ipython-input-4019085467.py:1: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='fcfdea6d-dc10-4160-a54e-bac1748b43da', metadata={}, page_content='درد قفسهٔ سینه می\u200cتواند ناشی از عوامل مختلفی مانند استرس، مشکلات عضلانی یا بیماری\u200cهای قلبی باشد'), np.float32(-200.45152)), (Document(id='3fa76f0e-c166-4297-8f82-5276613d4f5b', metadata={}, page_content='مصرف صحیح و منظم داروها طبق نسخهٔ پزشک برای اثربخشی درمان ضروری است.'), np.float32(-201.0786))]
  db.similarity_search_with_relevance_scores('اضطراب' , k=2)


[(Document(id='fcfdea6d-dc10-4160-a54e-bac1748b43da', metadata={}, page_content='درد قفسهٔ سینه می\u200cتواند ناشی از عوامل مختلفی مانند استرس، مشکلات عضلانی یا بیماری\u200cهای قلبی باشد'),
  np.float32(-200.45152)),
 (Document(id='3fa76f0e-c166-4297-8f82-5276613d4f5b', metadata={}, page_content='مصرف صحیح و منظم داروها طبق نسخهٔ پزشک برای اثربخشی درمان ضروری است.'),
  np.float32(-201.0786))]